# Part 1

In [1]:
#extract the table of Toronto neighborhoods from Wikipedia

import pandas as pd
import numpy as np

df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [2]:
#drop rows with no assigned Borough

neighborhood_df = df[~df.Borough.str.contains("Not assigned")]
neighborhood_df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [3]:
#get the number of rows in data frame

neighborhood_df.shape

(103, 3)

# Part 2

In [4]:
# @hidden_cell

#read CSV file of neighborhood latitude and longitude

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_72bb2cc2bf394f808e906b2846065a96 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='PuNa9bRFHZdD8v6PC8u5yn_cJKW4uGnCrdbbCBVm8dCz',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_72bb2cc2bf394f808e906b2846065a96.get_object(Bucket='segmentingandclusteringneighborho-donotdelete-pr-n57gifke3gdrz3',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

toronto_neighborhood = pd.read_csv(body)
toronto_neighborhood


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [5]:
#read CSV file of neighborhood latitude and longitude (hidden cell because it includes my credentials), saved as pandas dataframe "toronto_neighborhood"

toronto_neighborhood

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [6]:
#include the borough and neighborhood name in the toronto_neighborhood dataframe

toronto_longlat = pd.merge(neighborhood_df,toronto_neighborhood,on="Postal Code")
toronto_longlat

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Part 3

In [7]:
#import additional libraries

import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

!python3 -m pip install folium

import folium
from geopy.geocoders import Nominatim

from sklearn.cluster import KMeans

In [8]:
#explore the neighbourhood

address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_longlat['Latitude'], toronto_longlat['Longitude'], toronto_longlat['Borough'], toronto_longlat['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [9]:
#get k-means clusters of Toronto neighbourhoods

kclusters = 4
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_longlat[["Latitude", "Longitude"]])

kmeans.labels_[0:10] 

array([2, 2, 0, 3, 0, 1, 2, 3, 0, 0], dtype=int32)

In [10]:
#add cluster labels to dataframe

toronto_longlat.insert(0, 'Cluster Labels', kmeans.labels_)

In [11]:
toronto_longlat.head()

,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,43.753259,-79.329656
1,2,M4A,North York,Victoria Village,43.725882,-79.315572
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [12]:
#map the clusters
cluster_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# set colors for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# put markers
cluster_markers = []
for lat, lon, poi, cluster in zip(toronto_longlat['Latitude'], toronto_longlat['Longitude'], toronto_longlat['Neighbourhood'], toronto_longlat['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster_map)
       
cluster_map

## Analysis and Observation

The clusters are segmented into four areas: north, east, south, and west Toronto.
They are generally separated by the main roads/highways in red and gray.